## Task 3

### Logit


In [3]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
import statsmodels.api as sm

In [5]:
ins = pd.read_stata \
    ("C:/Users/Popov/Documents/Studies/NES_studies/Stata/EIO/HA/HA1/insured.dta")

In [8]:
ins.head(3)

,healthy,age,male,insured,deg_nd,deg_ged,deg_hs,deg_ba,deg_ma,deg_phd,...,married,selfemp,familysz,reg_ne,reg_mw,reg_so,reg_we,race_bl,race_ot,race_wht
0,1.0,31.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,31.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,1.0,54.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [9]:
ins.describe()

,healthy,age,male,insured,deg_nd,deg_ged,deg_hs,deg_ba,deg_ma,deg_phd,...,married,selfemp,familysz,reg_ne,reg_mw,reg_so,reg_we,race_bl,race_ot,race_wht
count,8802.000000,8802.000000,8802.000000,8802.000000,8802.000000,8802.000000,8802.000000,8802.000000,8802.000000,8802.000000,...,8802.000000,8802.000000,8802.000000,8802.000000,8802.000000,8802.000000,8802.000000,8802.000000,8802.000000,8802.000000
mean,0.928539,38.936832,0.526358,0.801182,0.127130,0.042490,0.503749,0.175983,0.059532,0.015337,...,0.617246,0.121677,3.093502,0.191093,0.229834,0.349352,0.229721,0.123040,0.041468,0.835492
std,0.257608,11.110823,0.499333,0.399134,0.333138,0.201716,0.500014,0.380827,0.236631,0.122898,...,0.486087,0.326931,1.559633,0.393184,0.420750,0.476793,0.420677,0.328502,0.199381,0.370757
min,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,30.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,39.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,48.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,4.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,62.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,14.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
# Define the dependent and independent variables
X = ins[['healthy', 'age', 'male', 'deg_nd', 'deg_ged', 'deg_hs', 'deg_ba', 'deg_ma', 'deg_phd', 
        'married', 'selfemp', 'familysz', 'reg_ne', 'reg_mw', 'reg_so', 'race_bl', 'race_ot']]
y = ins['insured']

# Add a constant 
X = sm.add_constant(X)

# Fit the logit
logit_model = sm.Logit(y, X).fit(cov_type='HC2')

# Print the summary
print(logit_model.summary())

Optimization terminated successfully.
         Current function value: 0.427051
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                insured   No. Observations:                 8802
Model:                          Logit   Df Residuals:                     8784
Method:                           MLE   Df Model:                           17
Date:                Mon, 30 Sep 2024   Pseudo R-squ.:                  0.1438
Time:                        13:10:41   Log-Likelihood:                -3758.9
converged:                       True   LL-Null:                       -4390.1
Covariance Type:                  HC2   LLR p-value:                5.516e-258
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5547      0.218      2.544      0.011       0.127       0.982
healthy        0.2562      0.

In [13]:
# Get the LaTeX output
latex_output = logit_model.summary().as_latex()

with open("logit_results.tex", "w") as f:
    f.write(latex_output)